# Create a meshed screen with a Gaussian beam

The screen is placed at the origin in the x-y-plane.<br>
The screen normal is pointing in the negative z direction (x - right, y - up).<br>
The beam is propagating in positive z direction.

if not yet available some libraries have to be installed :<br>
- gmsh (best installed globally through package management system)
- python3 -m pip install pygmsh --user
- python3 -m pip install vtk --user

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import vtk

import pygmsh
import numpy as np
from scipy import constants
import h5py

In [2]:
def CircularMesh(R, ratio=1.0, lcar=0.1):
    """
    Create a circular mesh with radius R around the origin [0,0,0].
    The normal vector is (0,0,1) in z-direction, the disk lies in the x-y-plane.
    
    The density of the grid is increased by ratio in the x-direction.
    The average cell dimensions are given by lcar. The average cell dimension in
    x-direction, thus, is lcar/ratio.
    
    returns:
    - an array of point corrdinates [3 float]
    - a connection list of triangles [3 int references into the array of points]
    """
    geom = pygmsh.built_in.Geometry()
    # we create the initial geometry as a streched ellipse to create
    # different scaling lengths (cell sizes) along the different axes
    p1 = geom.add_point([ratio, 0.0, 0.0], lcar)
    p2 = geom.add_point([0.0, 1.0, 0.0], lcar)
    p3 = geom.add_point([-ratio, 0.0, 0.0], lcar)
    p4 = geom.add_point([0.0, -1.0, 0.0], lcar)
    pc = geom.add_point([0.0, 0.0, 0.0], lcar)
    pa = geom.add_point([1.0, 0.0, 0.0], lcar)
    # the mesh is circumscribed with four elliptic arcs
    e1 = geom.add_ellipse_arc(start=p1, center=pc, point_on_major_axis=pa, end=p2)
    e2 = geom.add_ellipse_arc(start=p2, center=pc, point_on_major_axis=pa, end=p3)
    e3 = geom.add_ellipse_arc(start=p3, center=pc, point_on_major_axis=pa, end=p4)
    e4 = geom.add_ellipse_arc(start=p4, center=pc, point_on_major_axis=pa, end=p1)
    # these are combined into a line loop
    ll = geom.add_line_loop([e1,e2,e3,e4])
    geom.add_plane_surface(ll)
    # now we can create the mesh
    mesh = pygmsh.generate_mesh(geom, dim=2, verbose=False)
    # we reverse the streching by scaling the coordinates accordingly
    points = np.array([ p*R*[1.0/ratio,1,1] for p in mesh.points ])
    triangles = mesh.cells['triangle']
    return points, triangles

In [3]:
pts, tris = CircularMesh(R=0.04, ratio=1.0, lcar=0.1)
print("%d points" % len(pts))
print("%d triangles" % len(tris))
pos = [(pts[t[0]] + pts[t[1]] + pts[t[2]]) / 3.0 for t in tris]
NP = len(pos)
print("%d source positions" % NP)

459 points
848 triangles
848 source positions


In [4]:
def MeshArea(points, triangles):
    """
    Compute the area of all mesh cells
    """
    area=[]
    for i, t in enumerate(triangles):
        p1 = points[t[0]]
        p2 = points[t[1]]
        p3 = points[t[2]]
        r1 = p2-p1
        r2 = p3-p1
        area.append(0.5*np.linalg.norm(np.cross(r1,r2)))
    return np.array(area)

area = MeshArea(pts, tris)
print(np.min(area))
print(np.max(area))
print(np.sum(area))

3.07937892558e-06
1.16543167076e-05
0.00501847758487


In [5]:
def MeshNormals(points, triangles):
    """
    Compute the normal vectors of all mesh cells
    """
    normals=[]
    for i, t in enumerate(triangles):
        p1 = points[t[0]]
        p2 = points[t[1]]
        p3 = points[t[2]]
        r1 = p2-p1
        r2 = p3-p1
        n = np.cross(r1,r2)
        normals.append(n / np.linalg.norm(n))
    return np.array(normals)

normals = MeshNormals(pts, tris)
print(np.sum(normals, axis=0))

[   0.    0.  848.]


### define the beam parameters

In [6]:
# time step
dt=2.0e-14
NOTS=400

# wave parameters
tau = 1.0e-12
f = 2.0e12
print("f = %.2f THz" % (f*1e-12))
w0 = 0.005
lam = constants.c/f
print("λ = %.2f µm" % (lam*1e6))
zR = np.pi*w0*w0/lam
print("w0 = %.2f mm" % (1e3*w0))
print("Rayleigh range = %.3f m" % zR)


f = 2.00 THz
λ = 149.90 µm
w0 = 5.00 mm
Rayleigh range = 0.524 m


### compute the field traces for every point

In [7]:
# all points use the same timing grid
t = np.linspace(-NOTS/2*dt, (NOTS/2-1)*dt, NOTS)
# field (6 components) traces on screen
A = np.zeros((NP,NOTS,6))
# electric field amplitude in V/m
E0 = 2.0e7
# magnetic field amplitude in T
B0 = E0 / constants.c

for ip, p in enumerate(pos):
    r = np.sqrt(p[0]*p[0]+p[1]*p[1])
    # field strength
    osc = np.cos(2.0*np.pi*f*t) * np.exp(-r*r/(w0*w0)) * np.exp(-np.square(t)/(2.0*tau*tau))
    Ex = E0 * osc
    Ey = np.zeros_like(Ex)
    Ez = np.zeros_like(Ex)
    Bx = np.zeros_like(Ex)
    By = B0 * osc
    Bz = np.zeros_like(Ex)
    trace = np.array([Ex,Ey,Ez,Bx,By,Bz]).transpose()
    A[ip] = trace

In [8]:
# compute power density for plotting
Pz = []
for trace in A:
    EVec = trace[:,0:3]
    BVec = trace[:,3:6]
    SVec = np.cross(EVec, BVec) / constants.mu_0
    Pz.append((SVec.sum(axis=0))[2]*dt)

print("peak energy density = %.6f J/m²" % np.max(Pz))
print("total pulse energy = %.1f µJ" % (1e6*np.dot(area,Pz)))

peak energy density = 0.752298 J/m²
total pulse energy = 37.0 µJ


### display mesh

In [9]:
def ShowMesh(points, triangles, centers=[]):
    """
    Render a display of a mesh geometry using VTK.
    """
    # create a dataset for the triangle mesh
    pts = vtk.vtkPoints()
    for p in points:
        pts.InsertNextPoint(p)
    cells = vtk.vtkCellArray()
    for t in triangles:
        cells.InsertNextCell(3, t)
    meshData = vtk.vtkPolyData()
    meshData.SetPoints(pts)
    meshData.SetPolys(cells)
    # create a dataset for the center points
    if len(centers)>0:
        cpts = vtk.vtkPoints()
        for p in centers:
            cpts.InsertNextPoint(p)
        ccells = vtk.vtkCellArray()
        for i in range(len(centers)):
            ccells.InsertNextCell(1, [i])
        centerData = vtk.vtkPolyData()
        centerData.SetPoints(cpts)
        centerData.SetVerts(ccells)
    colors = vtk.vtkNamedColors()
    # map the triangle mesh into the scene
    meshMapper = vtk.vtkPolyDataMapper()
    meshMapper.SetInputData(meshData)
    meshActor = vtk.vtkActor()
    meshActor.SetMapper(meshMapper)
    meshActor.GetProperty().SetPointSize(5)
    meshActor.GetProperty().SetColor(colors.GetColor3d("Red"))
    meshActor.GetProperty().EdgeVisibilityOn()
    # map the center points into the scene
    if len(centers)>0:
        centerMapper = vtk.vtkPolyDataMapper()
        centerMapper.SetInputData(centerData)
        centerActor = vtk.vtkActor()
        centerActor.SetMapper(centerMapper)
        centerActor.GetProperty().SetPointSize(3)
        centerActor.GetProperty().SetColor(colors.GetColor3d("Blue"))
    # visualize the coordinate system
    axesActor = vtk.vtkAxesActor()
    # create a render window
    renderer = vtk.vtkRenderer()
    renderWindow = vtk.vtkRenderWindow()
    renderWindow.SetSize(800,600)
    renderWindow.AddRenderer(renderer)
    renderWindowInteractor = vtk.vtkRenderWindowInteractor()
    renderWindowInteractor.SetRenderWindow(renderWindow)
    renderWindowInteractor.Initialize()
    style = vtk.vtkInteractorStyleTrackballCamera()
    renderWindowInteractor.SetInteractorStyle(style)
    # add the actors to the scene
    renderer.AddActor(meshActor)
    if len(centers)>0: renderer.AddActor(centerActor)
    renderer.AddActor(axesActor)
    renderer.SetBackground(colors.GetColor3d("SlateGray"))
    # render and interact
    renderWindow.Render()
    renderWindowInteractor.Start()
    # now the interaction is running until we close the window
    # cleanup after closing the window
    del renderWindow
    del renderWindowInteractor

In [10]:
ShowMesh(pts,tris,pos)

In [27]:
def WriteMesh(filename, points, triangles, pos):
    """
    Write the meshed geometry to an HDF5 file.
    No time or field datasets are created in the file.
    """
    hf = h5py.File(filename, 'w')
    h5p = hf.create_dataset('MeshCornerPoints', data=points, dtype='f8')
    h5p.attrs['Ncp'] = len(points)
    h5p = hf.create_dataset('MeshTriangles', data=triangles, dtype='u4')
    h5p.attrs['Ntri'] = len(triangles)
    h5p = hf.create_dataset('ObservationPosition', data=pos)
    h5p.attrs['Np'] = len(pos)
    hf.close()

In [28]:
filename="CircularMesh.h5"
WriteMesh(filename, pts,tris,pos)

In [3]:
def ReadMesh(filename):
    """
    Read the meshed geometry from an HDF5 file.
    No time or field datasets are read.
    Return arrays for mesh corner points, triangle associations and center (trace) positions.
    """
    hdf = h5py.File(filename, "r")
    p = hdf['MeshCornerPoints']
    points = np.array(p)
    p = hdf['MeshTriangles']
    triangles = np.array(p)
    p = hdf['ObservationPosition']
    pos = np.array(p)
    hdf.close()
    return points, triangles, pos

In [4]:
pts, tris, pos = ReadMesh("CircularMesh.h5")
ShowMesh(pts,tris,pos)